In [9]:
from dataclasses import dataclass
import time
from tqdm import tqdm
from multiprocessing import Pool
from sklearn import preprocessing

from matplotlib.patches import FancyBboxPatch
import matplotlib.pyplot as plt
import matplotlib.path as mpath
from matplotlib import cm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.ticker import MultipleLocator, FormatStrFormatter,FuncFormatter
from matplotlib.dates import YearLocator, MonthLocator, DayLocator, HourLocator, MinuteLocator, SecondLocator, DateFormatter
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec
from matplotlib.legend_handler import HandlerPathCollection
from matplotlib.lines import Line2D
import matplotlib.patheffects as path_effects

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import obspy as op
from obspy import read,read_inventory, UTCDateTime, Stream, Trace
from obspy.clients.fdsn.client import Client
from obspy.signal.rotate import rotate_ne_rt
from obspy.geodetics import gps2dist_azimuth,kilometers2degrees
from obspy.taup import TauPyModel

import json
import glob
import os
import numpy as np
from itertools import combinations
import pandas as pd
from scipy.signal import spectrogram, detrend, resample,savgol_filter,decimate,hilbert
from scipy.stats import circmean, circstd,gaussian_kde
from kneed import KneeLocator

import pyarrow.feather as feather

import datetime

from sklearn.linear_model import LinearRegression,HuberRegressor,TheilSenRegressor
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler
from sklearn.metrics import silhouette_score

In [10]:
# ===========
# DIRECTORIES
# ===========

## ------------------------
## Directory of the catalog (.CSV file of the National Earthquake Information Center (NEIC))
## The file layout is defined as:
## time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource

XML_DIR = '/home/sysop/dados_posdoc/PROJETO_RSBR_15_YEARS/XML/'

## -----------------------
## Directory of the output (Figures and Feathers file)

ORIENTATION_OUTPUT = '/home/sysop/dados_posdoc/PROJETO_RSBR_15_YEARS/OUTPUT/'

# ==========
# PARAMETERS
# ==========

## -------------------------------------------------------------------
## Apply band-pass filtering to the seismograms using the range above:

PERIOD_BANDS = [0.02,0.5]

## ===================================================================================
## Default parameters to define the signal and noise windows used to estimate the SNR:

## ------------------------------------------------------------------------------
## Duration of the signal window before and after the P-wave arrival (in seconds)

TIME_WINDOW = 60

## -----------------------------------------------------------------------------
## Start time of the P-wave window for events (in seconds before P-wave arrival)

TIME_START_P_REGIONAL = 3

## -----------------------------------------------------------------------------------
## End time of the P-wave window for regional events (in seconds after P-wave arrival)

TIME_FINAL_P_REGIONAL = 12

## ---------------------------------------------
## Minimum earthquake magnitude to be considered

minmagnitude = 6

## -------------------------------------------------------------------------------------
## Minimum and maximum epicentral distance in degrees (GCARC: great-circle arc distance)

GCARC_MIN = 5
GCARC_MAX = 100

## -----------------
## Region parameters

LLCRNRLON_LARGE = -50
URCRNRLON_LARGE = -38
LLCRNRLAT_LARGE = -30
URCRNRLAT_LARGE = -12

## ---------
## Constants

ONEDAY = datetime.timedelta(days=1)

## ---------------
## MULTIPROCESSING

num_processes = 20

## --------------------------------------
## Generate a figure for each estimation?

VERBOSE = True

In [11]:
# Formatar os rótulos do eixo Y para exibir o símbolo de graus
def format_y_ticks(value, _):
    return f"{value:.0f}°"

In [12]:
def calculate_quartis_mask(orientations):
    # Estimating quartis
    Q1 = np.percentile(orientations, 25)
    Q3 = np.percentile(orientations, 75)
    IQR = Q3 - Q1
                            
    # Defining limits
    lower_bound = Q1 - 1 * IQR
    upper_bound = Q3 + 1 * IQR
    
    # Filter mask
    mask_good = (orientations >= lower_bound) & (orientations <= upper_bound)
    mask_outliers = ~mask_good  # Inverse

    return mask_good,mask_outliers

In [13]:
colnames = ['network', 'station', 'stla', 'stlo', 'evname', 'evla', 'evlo','evtime', 'evmag', 'evtype', 'evdp', 'distance', 'gcarc', 'baz','SNR', 'phi', 'theta','clock_error', 'quality', 'gain_HHN', 'gain_HHE', 'gain_HHZ','event_class']

In [16]:
STATION_LST = sorted([i.split('/')[-1].split('.')[:2] for i in glob.glob(XML_DIR+'BR.CZ*')])

In [17]:
STATION_LST

[['BR', 'CZSB']]

In [18]:
plt.rcParams.update({'font.size': 14})  # Define o tamanho global da fonte

for net_sta in tqdm(STATION_LST, total=len(STATION_LST), desc='Station'):
    
    net = net_sta[0]
    sta = net_sta[1]

    colnames = ['network', 'station','evtime','SNR', 'phi', 'theta','clock_error', 'quality', 'gain_HHN', 'gain_HHE', 'gain_HHZ','event_class']
    
    feather_files_lst = [pd.read_feather(i,columns=colnames) for i in glob.glob(ORIENTATION_OUTPUT+'FEATHER_FILES/*/*'+sta+'*/*')]
    
    station_df = pd.concat(feather_files_lst)
    station_df['year_month'] = station_df['evtime'].dt.to_period('M').astype(str)
    station_df['year_month'] = pd.to_datetime(station_df['year_month'], format='%Y-%m').dt.to_period('M')
    
    df_sta = station_df[station_df['station'] == sta].copy()

    # ------------------------------------
    # Data
    
    orientations_all_good = df_sta[df_sta['quality'] == 'good']['theta'].values # Orientations
    time_all_good = df_sta[df_sta['quality'] == 'good']['evtime'].values  # Time in datetime
    time_all_stamp = df_sta[df_sta['quality'] == 'good']['evtime'].apply(lambda x: int(x.timestamp()))  # Time in Timestamp

    if len(orientations_all_good) > 50:

        # ================================= #
        # START: DBSCAN clusters estimation #
        # ================================= #
        
        # ------------------------------------
        # This Scaler removes the median and scales the data according to the quantile range (defaults to IQR: Interquartile Range). 
        # The IQR is the range between the 1st quartile (25th quantile) and the 3rd quartile (75th quantile).
    
        data_all = np.array([time_all_stamp,orientations_all_good]).T
        scaler = RobustScaler()
        scaler.fit(data_all)
        data_scale = scaler.transform(data_all)

        # The size of the radius is specified by the distance threshold parameter (epsilon)
        
        eps_range = np.arange(0.23,0.28,0.005)
        per_samples = 20 # percentage of total de samples per group

        # -----------------------------
        # Silhouette DBSCAN estimation
        
        eps_lst = []
        silhouette_score_lst = []

        for n_eps in eps_range:
            try:
                clustering = DBSCAN(eps=round(n_eps,3), min_samples=len(orientations_all_good)//per_samples).fit(data_scale)
                ss = silhouette_score(data_scale, clustering.fit_predict(data_scale))
                
                eps_lst.append(n_eps)
                silhouette_score_lst.append(ss)
            except:
                pass 

        if len(silhouette_score_lst) > 0:
            # ------------------------------------    
            # Elbow DBSCAN estimation
            
            kn = KneeLocator(eps_lst, silhouette_score_lst, curve='concave',polynomial_degree=3,S=5)
            elbow_point = kn.knee
            
            if not elbow_point:
                kn = KneeLocator(eps_lst, silhouette_score_lst, curve='concave',polynomial_degree=3)
                elbow_point = kn.knee
        
            silhouette_score_elbow_point = silhouette_score_lst[eps_lst.index(elbow_point)]


            # ------------------------
            # DBSCAN clustering result

            clustering = DBSCAN(eps=round(elbow_point,2), min_samples=len(orientations_all_good)//per_samples).fit(data_scale)
            
            df_sta['class'] = -10  # standart value
            df_sta.loc[df_sta['quality'] == 'good', 'class'] = clustering.labels_
            unique_labels = list(set(clustering.labels_))

            if silhouette_score_elbow_point > 0.22 and len(unique_labels) < 6:

                colors = [mcolors.to_rgba(plt.cm.Spectral(each),alpha=0.5) for each in np.linspace(0, 1, len(unique_labels))]
                label_dbscan = ['g'+str(1+w)+': ' if w != -1 else 'ol: ' for w in unique_labels]
                
                label_dbscan_lst_end = [] 
                for idx,labe in enumerate(unique_labels):
                    label_dbscan_lst_end.append(label_dbscan[idx]+str(len(df_sta[(df_sta['quality'] == 'good') & (df_sta['class'] == labe)]['theta'].values)))
            
                # =============================== #
                # END: DBSCAN clusters estimation #
                # =============================== #
            
                # Cria o range de meses como Periods e converte para datetime (timestamp)
                years = pd.period_range(start=df_sta['year_month'].min(), end=df_sta['year_month'].max(), freq='M')
                
                fig = plt.figure(figsize=(10, 10))
                gs = gridspec.GridSpec(3, 2, width_ratios=[10,1], height_ratios=[1,10,1],hspace=0.05, wspace=0.05)
            
                # Definindo os eixos
                ax0 = fig.add_subplot(gs[0, 0])  #  Number of events axis
                ax1 = fig.add_subplot(gs[1, 0],sharex=ax0)  # Orientation axis
                ax2 = fig.add_subplot(gs[2, 0],sharex=ax0)  # Histogram axis
                ax3 = fig.add_subplot(gs[1, 1],sharey=ax1)  # KDE axis

                if silhouette_score_elbow_point:
                    ax0.annotate('ss:'+str(round(silhouette_score_elbow_point,2)), (pd.to_datetime(df_sta['evtime'].values).max(), +15),fontsize=10, va='center', ha='center',bbox=dict(boxstyle="round", fc="white", ec='k', alpha=0.5))

                for ye in years:
                    # Filtering according to Period (year_month)
                    df_sta_year = df_sta[df_sta['year_month'] == ye]
            
                    ye_num = mdates.date2num(ye)  # converte timestamp para número

                    # --------- #
                    # Gain plot #
                    # --------- #

                    gain_HHE_all = df_sta_year['gain_HHE'].values.mean() # Gain HHE
                    gain_HHN_all = df_sta_year['gain_HHN'].values.mean() # Gain HHN
                    gain_HHZ_all = df_sta_year['gain_HHZ'].values.mean() # Gain HHZ

                    g1 = ax2.scatter(ye_num,np.log(gain_HHE_all/gain_HHN_all),c='k',marker='p',edgecolor='none',alpha=0.25,label='E/N')
                    g2 = ax2.scatter(ye_num,np.log(gain_HHE_all/gain_HHZ_all),c='k',marker='>',edgecolor='none',alpha=0.25,label='E/Z')
                    g3 = ax2.scatter(ye_num,np.log(gain_HHN_all/gain_HHZ_all),c='k',marker='^',edgecolor='none',alpha=0.25,label='N/Z')
                    
                    if df_sta_year[df_sta_year['quality'] == 'good']['theta'].empty:
                        orientations_bad = df_sta_year[df_sta_year['quality'] == 'bad']['theta'].values
                        snr_bad = df_sta_year[df_sta_year['quality'] == 'bad']['SNR'].abs().values

                        ax0.bar(ye_num, len(orientations_bad), color='gray', width=20, alpha=0.25,zorder=-10)
                        ax1.scatter([ye_num]*len(orientations_bad), orientations_bad, marker='.', c='gray',s=snr_bad*10, alpha=0.05, ec='k', label='bad')
                    
                    else:
                        
                        orientations_good = df_sta_year[df_sta_year['quality'] == 'good']['theta'].values
                        orientations_bad = df_sta_year[df_sta_year['quality'] == 'bad']['theta'].values

                        snr_good = df_sta_year[df_sta_year['quality'] == 'good']['SNR'].abs().values
                        snr_bad = df_sta_year[df_sta_year['quality'] == 'bad']['SNR'].abs().values
                        
                        ax0.bar(ye_num, len(orientations_bad), color='gray', width=20, alpha=0.25,zorder=-9)
                        ax1.scatter([ye_num]*len(orientations_bad), orientations_bad, marker='.', c='gray',s=snr_bad*10, alpha=0.05, ec='k', label='bad')

                        for uni,col in zip(unique_labels,colors):
                            orientations_good_cluster = df_sta_year[(df_sta_year['quality'] == 'good') & (df_sta_year['class'] == uni)]['theta'].values
                            snr_good_cluster = df_sta_year[(df_sta_year['quality'] == 'good') & (df_sta_year['class'] == uni)]['SNR'].abs().values

                            ax0.bar(ye_num, len(orientations_good_cluster), color=col, width=10, alpha=0.5,zorder=1)
                            ax1.scatter([ye_num]*len(orientations_good_cluster), orientations_good_cluster, marker='.', color=col,s=snr_good_cluster*10, alpha=0.25, ec='k')                          
        
                            if uni != -1:

                                ax1.scatter([ye_num]*len(orientations_good_cluster), orientations_good_cluster, marker='.', color=col,s=snr_good_cluster*10, alpha=0.5, ec='k')
                                ax1.annotate(f"{round(np.mean(df_sta[(df_sta['quality'] == 'good') & (df_sta['class'] == uni)]['theta'].values), 1)}±{round(np.std(df_sta[(df_sta['quality'] == 'good') & (df_sta['class'] == uni)]['theta'].values), 2)}°",(pd.to_datetime(df_sta[(df_sta['quality'] == 'good') & (df_sta['class'] == uni)]['evtime'].values).mean(), round(np.mean(df_sta[(df_sta['quality'] == 'good') & (df_sta['class'] == uni)]['theta'].values), 1)),fontsize=12, va='center', ha='center',path_effects=[path_effects.Stroke(linewidth=3, foreground='white'), path_effects.Normal()])
                                ax0.bar(ye_num, len(orientations_good_cluster), color=col, width=20,zorder=10)


                bad_values = len(df_sta[df_sta['quality'] == 'bad']['theta'].values)
                label_dbscan_lst_end.append('bd: '+str(bad_values))

                colors.append(mcolors.to_rgba('gray',alpha=0.25))  # add 'gray' as RGBA
                
                # Personalised handles
                handles = [Line2D([0], [0], marker='o', color='w', label=grupo,markerfacecolor=cor,markeredgecolor='k',markersize=8) for grupo, cor in zip(label_dbscan_lst_end, colors)]

                # ------------------------------ #
                # Kernel density estimation plot #
                # ------------------------------ #
                
                kde = gaussian_kde(orientations_all_good)
                x_vals = np.linspace(min(orientations_all_good), max(orientations_all_good), 1000)
                density = kde(x_vals)
                        
                ax3.plot(density,x_vals, '-k')
            
                # Histogram parameters

                ax0.yaxis.set_major_locator(MultipleLocator(5))
                ax0.tick_params(axis="x", which='both', labelbottom=False, labeltop=False, rotation=30)
                ax0.tick_params(axis="y", which='both', labelright=False, labelleft=True, left=True, right=True)
                ax0.set_title(f'{net}.{sta}', fontsize=20)
                ax0.set_ylim(0, 20)
                ax0.set_ylabel("n")
                ax0.grid(True)
                ax0.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
                ax0.xaxis.set_major_locator(mdates.MonthLocator(interval=12))
                ax0.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))

                # Orientation parameters
                
                ax1.set_ylabel(r'Orientation($\theta$)')
                ax1.set_ylim(-200, 200)
                ax1.yaxis.set_major_locator(MultipleLocator(40))
                ax1.yaxis.set_minor_locator(MultipleLocator(10))
                ax1.yaxis.set_major_formatter(FuncFormatter(format_y_ticks))
                ax1.grid(True)
                ax1.tick_params(axis="x", which='both', labelbottom=False, labeltop=False, top=True, rotation=30)
                ax1.tick_params(axis="y", which='both', labelright=False, labelleft=True, left=True, right=True)
                ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
                ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=12))
                ax1.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))
                ax1.legend(handles=handles,loc='lower right',ncols=3)
                
                # Gain parameters
                
                ax2.figure.legend(handles=[g1, g2, g3],loc='center',bbox_to_anchor=(0.86, 0.14),frameon=False,ncol=1,fontsize=10,borderaxespad=0.)
                ax2.tick_params(axis="x", which='both', labelbottom=True, labeltop=False, rotation=30)
                ax2.tick_params(axis="y", which='both', labelright=False, labelleft=True, left=True, right=True)
                ax2.grid(True)
                ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
                ax2.xaxis.set_major_locator(mdates.MonthLocator(interval=12))
                ax2.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))
                ax2.set_ylabel("Gain")

                # KDE parameters
                
                ax3.set_xlabel('KDE')
                ax3.xaxis.set_label_position('top')
                ax3.yaxis.set_major_formatter(FuncFormatter(format_y_ticks))
                ax3.tick_params(axis="x", which='both', labelbottom=False, labeltop=False, rotation=30)
                ax3.tick_params(axis="y", which='both', labelright=True, labelleft=False,labelbottom=False, labeltop=False, left=True, right=True,top=True, bottom=False)
                ax3.grid(True)
                # Saving the figure
                
                output_figure_ORIENTATION = ORIENTATION_OUTPUT + 'ORIENTATION_FIGURES/FINAL_RESULT/'+net+'/'
                os.makedirs(output_figure_ORIENTATION, exist_ok=True)
                fig.savefig(output_figure_ORIENTATION + f'ORIENTATION_TOTAL_{net}_{sta}.png',facecolor='w',dpi=300)
                plt.close()
            
            else:
            
                # ---------
                # NO DBSCAN
      
                # -----------------------
                # Outlier analysis result
                        
                df_sta['class'] = -10  # standart value
                             
                # Filter mask
                mask_good,mask_outliers = calculate_quartis_mask(orientations_all_good)
    
                df_sta.loc[df_sta['quality'] == 'good', 'class'] = [1 if i == True else -1 for i in mask_good]
            
                # Creating mounth range datetime (timestamp) Periods
                years = pd.period_range(start=df_sta['year_month'].min(), end=df_sta['year_month'].max(), freq='M')
                
                fig = plt.figure(figsize=(10, 10))
                gs = gridspec.GridSpec(3, 2, width_ratios=[10,1], height_ratios=[1,10,1],hspace=0.05, wspace=0.05)
            
                # Definindo os eixos
                ax0 = fig.add_subplot(gs[0, 0])  #  Number of events axis
                ax1 = fig.add_subplot(gs[1, 0],sharex=ax0)  # Orientation axis
                ax2 = fig.add_subplot(gs[2, 0],sharex=ax0)  # Histogram axis
                ax3 = fig.add_subplot(gs[1, 1],sharey=ax1)  # KDE axis
                
                if silhouette_score_elbow_point:
                    ax0.annotate('ss:'+str(round(silhouette_score_elbow_point,2)), (pd.to_datetime(df_sta['evtime'].values).max(), +15),fontsize=10, va='center', ha='center',bbox=dict(boxstyle="round", fc="white", ec='k', alpha=0.5))
                
                label_handles_bad = []
                label_handles_out = []
                label_handles_dat = []
                
                for ye in years:
                    # Filtering according to Period (year_month)
                    df_sta_year = df_sta[df_sta['year_month'] == ye]
            
                    ye_num = mdates.date2num(ye)  # converte timestamp para número
                    
                    # --------- #
                    # Gain plot #
                    # --------- #

                    gain_HHE_all = df_sta_year['gain_HHE'].values.mean() # Gain HHE
                    gain_HHN_all = df_sta_year['gain_HHN'].values.mean() # Gain HHN
                    gain_HHZ_all = df_sta_year['gain_HHZ'].values.mean() # Gain HHZ

                    g1 = ax2.scatter(ye_num,np.log(gain_HHE_all/gain_HHN_all),c='k',marker='p',edgecolor='none',alpha=0.25,label='E/N')
                    g2 = ax2.scatter(ye_num,np.log(gain_HHE_all/gain_HHZ_all),c='k',marker='>',edgecolor='none',alpha=0.25,label='E/Z')
                    g3 = ax2.scatter(ye_num,np.log(gain_HHN_all/gain_HHZ_all),c='k',marker='^',edgecolor='none',alpha=0.25,label='N/Z')
                    
                    if df_sta_year[df_sta_year['quality'] == 'good']['theta'].empty:
                        orientations_bad = df_sta_year[df_sta_year['quality'] == 'bad']['theta'].values
                        snr_bad = df_sta_year[df_sta_year['quality'] == 'bad']['SNR'].abs().values

                        ax0.bar(ye_num, len(orientations_bad), color='gray', width=20, alpha=0.25,zorder=-10)
                        ax1.scatter([ye_num]*len(orientations_bad), orientations_bad, marker='.', c='gray',s=snr_bad*10, alpha=0.05, ec='k', label='out')
    
                        label_handles_bad.append(len(orientations_bad))
    
                    else:
                        orientations_good = df_sta_year[(df_sta_year['quality'] == 'good') & (df_sta_year['class'] == 1)]['theta'].values
                        orientations_out = df_sta_year[(df_sta_year['quality'] == 'good') & (df_sta_year['class'] == -1)]['theta'].values
                        orientations_bad = df_sta_year[df_sta_year['quality'] == 'bad']['theta'].values
                                    
                        snr_good = df_sta_year[(df_sta_year['quality'] == 'good') & (df_sta_year['class'] == 1)]['SNR'].abs().values
                        snr_out = df_sta_year[(df_sta_year['quality'] == 'good') & (df_sta_year['class'] == -1)]['SNR'].abs().values
                        snr_bad = df_sta_year[df_sta_year['quality'] == 'bad']['SNR'].abs().values
    
                        # Bad orientations
                        
                        ax0.bar(ye_num, len(orientations_bad), color='gray', width=20, alpha=0.25,zorder=-9)
                        ax1.scatter([ye_num]*len(orientations_bad), orientations_bad, marker='.', c='gray',s=snr_bad*10, alpha=0.05, ec='k', label='out')
                        
                        label_handles_bad.append(len(orientations_bad))

                        # Outliers orientations

                        ax0.bar(ye_num, len(orientations_out), color='mediumpurple', width=20, alpha=0.5,zorder=-5)
                        ax1.scatter([ye_num]*len(orientations_out), orientations_out, marker='.', c='mediumpurple',s=snr_out*10, alpha=0.25, ec='k')
                        
                        # Good orientations
                        
                        ax0.bar(ye_num, len(orientations_good), color='darkred', width=20, alpha=0.75,zorder=1)                                       
                        ax1.scatter([ye_num]*len(orientations_good), orientations_good, marker='.', c='darkred',s=snr_good*10, alpha=0.75, ec='k')
                        
                        label_handles_out.append(len(orientations_out))
                        label_handles_dat.append(len(orientations_good))

            
                ax1.annotate(f"{round(np.mean(df_sta[(df_sta['quality'] == 'good') & (df_sta['class'] == 1)]['theta'].values), 1)}±{round(np.std(df_sta[(df_sta['quality'] == 'good') & (df_sta['class'] == 1)]['theta'].values), 2)}°",(pd.to_datetime(df_sta[(df_sta['quality'] == 'good') & (df_sta['class'] == 1)]['evtime'].values).mean(), round(np.mean(df_sta[(df_sta['quality'] == 'good') & (df_sta['class'] == 1)]['theta'].values), 1)),fontsize=12, va='center', ha='center',path_effects=[path_effects.Stroke(linewidth=3, foreground='white'), path_effects.Normal()])
                       
                label_handles = ['bd: '+str(sum(label_handles_bad)),'ol: '+str(sum(label_handles_out)),'g1: '+str(sum(label_handles_dat))]
                alphas = [0.05,0.25,0.75] 
                colors = ['gray','mediumpurple','darkred']
                colors_with_alpha = [mcolors.to_rgba(cor, alpha=a) for cor, a in zip(colors, alphas)]
                
                # Personalised handles
                handles = [Line2D([0], [0], marker='o',color='w', label=grupo,markerfacecolor=cor,markeredgecolor='k',markersize=8) for grupo, cor in zip(label_handles, colors_with_alpha)]

                # ------------------------------ #
                # Kernel density estimation plot #
                # ------------------------------ #
                
                kde = gaussian_kde(orientations_all_good)
                x_vals = np.linspace(min(orientations_all_good), max(orientations_all_good), 1000)
                density = kde(x_vals)
                        
                ax3.plot(density,x_vals, '-k')
            
                # Histogram parameters

                ax0.yaxis.set_major_locator(MultipleLocator(5))
                ax0.tick_params(axis="x", which='both', labelbottom=False, labeltop=False, rotation=30)
                ax0.tick_params(axis="y", which='both', labelright=False, labelleft=True, left=True, right=True)
                ax0.set_title(f'{net}.{sta}', fontsize=20)
                ax0.set_ylim(0, 20)
                ax0.set_ylabel("n")
                ax0.grid(True)
                ax0.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
                ax0.xaxis.set_major_locator(mdates.MonthLocator(interval=12))
                ax0.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))

                # Orientation parameters
                
                ax1.set_ylabel(r'Orientation($\theta$)')
                ax1.set_ylim(-200, 200)
                ax1.yaxis.set_major_locator(MultipleLocator(40))
                ax1.yaxis.set_minor_locator(MultipleLocator(10))
                ax1.yaxis.set_major_formatter(FuncFormatter(format_y_ticks))
                ax1.grid(True)
                ax1.tick_params(axis="x", which='both', labelbottom=False, labeltop=False, top=True, rotation=30)
                ax1.tick_params(axis="y", which='both', labelright=False, labelleft=True, left=True, right=True)
                ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
                ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=12))
                ax1.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))
                ax1.legend(handles=handles,loc='lower right',ncols=3)
                
                # Gain parameters
                
                ax2.figure.legend(handles=[g1, g2, g3],loc='center',bbox_to_anchor=(0.86, 0.14),frameon=False,ncol=1,fontsize=10,borderaxespad=0.)
                ax2.tick_params(axis="x", which='both', labelbottom=True, labeltop=False, rotation=30)
                ax2.tick_params(axis="y", which='both', labelright=False, labelleft=True, left=True, right=True)
                ax2.grid(True)
                ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
                ax2.xaxis.set_major_locator(mdates.MonthLocator(interval=12))
                ax2.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))
                ax2.set_ylabel("Gain")

                # KDE parameters
                
                ax3.set_xlabel('KDE')
                ax3.xaxis.set_label_position('top')
                ax3.yaxis.set_major_formatter(FuncFormatter(format_y_ticks))
                ax3.tick_params(axis="x", which='both', labelbottom=False, labeltop=False, rotation=30)
                ax3.tick_params(axis="y", which='both', labelright=True, labelleft=False,labelbottom=False, labeltop=False, left=True, right=True,top=True, bottom=False)
                ax3.grid(True)

                # Salvando a figura
                output_figure_ORIENTATION = ORIENTATION_OUTPUT + 'ORIENTATION_FIGURES/FINAL_RESULT/'+net+'/'
                os.makedirs(output_figure_ORIENTATION, exist_ok=True)
                fig.savefig(output_figure_ORIENTATION + f'ORIENTATION_TOTAL_{net}_{sta}.png',facecolor='w',dpi=300)
                plt.close()
    else:
        print('station:',sta,'size:',len(df_sta))


Station:   0%|                                                                                                                                                                               | 0/1 [00:00<?, ?it/s]/tmp/ipykernel_8417/3315684642.py:122: RuntimeWarning: Mean of empty slice.
  gain_HHE_all = df_sta_year['gain_HHE'].values.mean() # Gain HHE
/home/sysop/Programs/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_8417/3315684642.py:123: RuntimeWarning: Mean of empty slice.
  gain_HHN_all = df_sta_year['gain_HHN'].values.mean() # Gain HHN
/tmp/ipykernel_8417/3315684642.py:124: RuntimeWarning: Mean of empty slice.
  gain_HHZ_all = df_sta_year['gain_HHZ'].values.mean() # Gain HHZ
/tmp/ipykernel_8417/3315684642.py:122: RuntimeWarning: Mean of empty slice.
  gain_HHE_all = df_sta_year['gain_HHE'].values.mean() # Gain HHE
/home/sysop/Programs/anaconda3/lib/py

In [29]:
plt.rcParams.update({'font.size': 14})  # Define o tamanho global da fonte

for sta in tqdm(STATION_LST, total=len(STATION_LST), desc='Station'):
    df_sta = station_df[station_df['station'] == sta].copy()
    net = df_sta['network'].unique().tolist()[0]

    # Cria o range de meses como Periods e converte para datetime (timestamp)
    years = pd.period_range(start=df_sta['year_month'].min(), end=df_sta['year_month'].max(), freq='M')

    # Criação da figura
    fig = plt.figure(figsize=(10, 10))
    gs = gridspec.GridSpec(2, 1, height_ratios=[10, 1], hspace=0.01)

    ax1 = fig.add_subplot(gs[0])  # orientações
    ax2 = fig.add_subplot(gs[1], sharex=ax1)  # número de eventos

    for ye in years:
        # Converte para Period para filtrar
        df_sta_year = df_sta[df_sta['year_month'] == ye]

        ye_num = mdates.date2num(ye)  # converte timestamp para número

        if df_sta_year[df_sta_year['quality'] == 'good']['theta'].empty:
            orientations_bad = df_sta_year[df_sta_year['quality'] == 'bad']['clock_error'].values
            snr_bad = df_sta_year[df_sta_year['quality'] == 'bad']['SNR'].abs().values

            a = ax1.scatter([ye_num]*len(orientations_bad), orientations_bad, marker='.', c='gray',s=snr_bad*10, alpha=0.05, ec='k', label='bad')
            ax2.bar(ye_num, len(orientations_bad), color='gray', width=20, alpha=0.5)
        else:
            orientations_good = df_sta_year[df_sta_year['quality'] == 'good']['clock_error'].values
            orientations_bad = df_sta_year[df_sta_year['quality'] == 'bad']['clock_error'].values
            snr_good = df_sta_year[df_sta_year['quality'] == 'good']['SNR'].abs().values
            snr_bad = df_sta_year[df_sta_year['quality'] == 'bad']['SNR'].abs().values

            mean_sta = round(np.mean(orientations_good), 1)
            std_sta = round(np.std(orientations_good), 2)

            ax1.scatter([ye_num]*len(orientations_bad), orientations_bad, marker='.', c='gray',s=snr_bad*10, alpha=0.05, ec='k', label='bad')
            b = ax1.scatter([ye_num]*len(orientations_good), orientations_good, marker='.', c='k',s=snr_good*10, alpha=0.5, ec='k', label='good')

            ax1.boxplot(orientations_good, positions=[ye_num], sym='', showmeans=False, widths=20, bootstrap=10000)

            ax2.bar(ye_num, len(orientations_bad), color='gray', width=20, alpha=0.5)
            ax2.bar(ye_num, len(orientations_good), color='k', edgecolor='k', width=10)

    ax1.set_ylabel('Clock error (s)')
    ax1.set_xlabel('YYYY-MM')
    ax1.set_ylim(-100,100)
    
    ax1.yaxis.set_major_locator(MultipleLocator(20))
    ax1.yaxis.set_minor_locator(MultipleLocator(10))
    ax1.grid(True)
    ax1.tick_params(axis="x", which='both', labelbottom=False, labeltop=True, rotation=30)
    ax1.tick_params(axis="y", which='both', labelright=True, labelleft=True, left=True, right=True)
    ax1.set_title(f'{net}.{sta}', fontsize=20)
    ax1.legend(handles=[a,b])

    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=12))
    ax1.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))
    
    ax2.yaxis.set_major_locator(MultipleLocator(5))
    ax2.tick_params(axis="x", which='both', labelbottom=True, labeltop=False, rotation=30)
    ax2.tick_params(axis="y", which='both', labelright=True, labelleft=True, left=True, right=True)
    ax2.set_ylim(0, 20)
    ax2.set_ylabel("n")
    ax2.grid(True)
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    ax2.xaxis.set_major_locator(mdates.MonthLocator(interval=12))
    ax2.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))

    output_figure_ORIENTATION = ORIENTATION_OUTPUT+'TIMING_FIGURES/FINAL_RESULT/'
    os.makedirs(output_figure_ORIENTATION,exist_ok=True)
    fig.savefig(output_figure_ORIENTATION+'TIMING_TOTAL_'+sta+'.png',dpi=300)
    plt.close()


Station:   0%|                                            | 0/1 [00:00<?, ?it/s]


ValueError: ('Lengths must match to compare', (605,), (2,))

In [ ]:
# Criar o semestre como 1 (jan-jun) ou 2 (jul-dez)
station_df['semester'] = station_df['evtime'].dt.month.map(lambda m: 1 if m <= 6 else 2)

# Criar coluna final combinando ano e semestre
station_df['year_semester'] = station_df['evtime'].dt.year.astype(str) + '-' + station_df['semester'].astype(str).str.zfill(2)

In [ ]:
station_df['year_semester'].head(1)

In [ ]:
plt.rcParams.update({'font.size': 12})  # Define o tamanho global da fonte

for sta in tqdm(STATION_LST, total=len(STATION_LST), desc='Station'):
    df_sta = station_df[station_df['station'] == sta].copy()
    net = df_sta['network'].unique().tolist()[0]

    # Cria o range de meses como Periods e converte para datetime (timestamp)
    years = list(range(2010, 2026))  # até 2025
    semesters = ['01', '02']
    
    # Construir todos os semestres no formato 'YYYY-SS'
    semesters_years = [f"{year}-{semester}" for year in years for semester in semesters]

    semestre_to_num = {sem: i for i, sem in enumerate(semesters_years)}

    # Criação da figura
    fig = plt.figure(figsize=(10, 10))
    gs = gridspec.GridSpec(2, 1, height_ratios=[10, 1], hspace=0.01)

    ax1 = fig.add_subplot(gs[0])  # orientações
    ax2 = fig.add_subplot(gs[1], sharex=ax1)  # número de eventos

    for ye_num in semesters_years:
        # Converte para Period para filtrar
        df_sta_year = df_sta[df_sta['year_semester'] == ye_num]

        ye_num = semestre_to_num[ye_num]

        if df_sta_year[df_sta_year['quality'] == 'good']['theta'].empty:
            orientations_bad = df_sta_year[df_sta_year['quality'] == 'bad']['theta'].values
            snr_bad = df_sta_year[df_sta_year['quality'] == 'bad']['SNR'].abs().values

            a = ax1.scatter([ye_num]*len(orientations_bad), orientations_bad, marker='.', c='gray',s=snr_bad*10, alpha=0.05, ec='k', label='bad')
            ax2.bar(ye_num, len(orientations_bad), color='gray', width=0.5, alpha=0.5)
        else:
            orientations_good = df_sta_year[df_sta_year['quality'] == 'good']['theta'].values
            orientations_bad = df_sta_year[df_sta_year['quality'] == 'bad']['theta'].values
            snr_good = df_sta_year[df_sta_year['quality'] == 'good']['SNR'].abs().values
            snr_bad = df_sta_year[df_sta_year['quality'] == 'bad']['SNR'].abs().values

            mean_sta = round(circmean(np.radians(orientations_good)), 1)
            std_sta = round(circstd(np.radians(orientations_good)), 2)

            ax1.scatter([ye_num]*len(orientations_bad), orientations_bad, marker='.', c='gray',s=snr_bad*10, alpha=0.05, ec='k', label='bad')
            b = ax1.scatter([ye_num]*len(orientations_good), orientations_good, marker='.', c='k',s=snr_good*10, alpha=0.5, ec='k', label='good')

            ax1.annotate(f'{mean_sta}±{abs(std_sta)}°', (ye_num, 95), fontsize=10, rotation=90,va='center', ha='center',bbox=dict(boxstyle="round", fc="white", ec='k', alpha=0.5))
            ax1.boxplot(orientations_good, positions=[ye_num], sym='', showmeans=False, widths=0.25, bootstrap=10000)

            ax2.bar(ye_num, len(orientations_bad), color='gray', width=0.5, alpha=0.5)
            ax2.bar(ye_num, len(orientations_good), color='k', edgecolor='k', width=0.1)


    # Ajustes visuais
    ax1.set_ylabel(r'Orientation($\theta$)')
    ax1.set_xlabel('YYYY-SS')
    ax1.set_ylim(-180, 180)
    ax1.yaxis.set_major_locator(MultipleLocator(40))
    ax1.yaxis.set_minor_locator(MultipleLocator(10))
    ax1.yaxis.set_major_formatter(FuncFormatter(format_y_ticks))
    ax1.grid(True)

    # Depois de plotar todos, ajustar o eixo X
    ax1.set_xticks(list(semestre_to_num.values()))
    ax1.set_xticklabels(list(semestre_to_num.keys()))
        
    ax1.tick_params(axis="x", which='both', labelbottom=False, labeltop=True, rotation=90)
    ax1.tick_params(axis="y", which='both', labelright=True, labelleft=True, left=True, right=True)
    ax1.set_title(f'{net}.{sta}', fontsize=20)
    ax1.legend(handles=[a,b])

    # Depois de plotar todos, ajustar o eixo X
    ax2.set_xticks(list(semestre_to_num.values()))
    ax2.set_xticklabels(list(semestre_to_num.keys()))
    ax2.yaxis.set_major_locator(MultipleLocator(15))
    ax2.tick_params(axis="x", which='both', labelbottom=True, labeltop=False, rotation=90)
    ax2.tick_params(axis="y", which='both', labelright=True, labelleft=True, left=True, right=True)
    ax2.set_ylim(0, 50)
    ax2.set_ylabel("n")
    ax2.grid(True)

    # Salvando a figura
    output_figure_ORIENTATION = ORIENTATION_OUTPUT + 'ORIENTATION_FIGURES/FINAL_RESULT/'
    os.makedirs(output_figure_ORIENTATION, exist_ok=True)
    fig.savefig(output_figure_ORIENTATION + f'ORIENTATION_TOTAL_{sta}.png', dpi=300)
    plt.close()


In [ ]:
plt.rcParams.update({'font.size': 12})  # Define o tamanho global da fonte

for sta in tqdm(STATION_LST, total=len(STATION_LST), desc='Station'):
    df_sta = station_df[station_df['station'] == sta].copy()
    net = df_sta['network'].unique().tolist()[0]

    # Cria o range de meses como Periods e converte para datetime (timestamp)
    years = list(range(2010, 2026))  # até 2025
    semesters = ['01', '02']
    
    # Construir todos os semestres no formato 'YYYY-SS'
    semesters_years = [f"{year}-{semester}" for year in years for semester in semesters]

    semestre_to_num = {sem: i for i, sem in enumerate(semesters_years)}

    # Criação da figura
    fig = plt.figure(figsize=(10, 10))
    gs = gridspec.GridSpec(2, 1, height_ratios=[10, 1], hspace=0.01)

    ax1 = fig.add_subplot(gs[0])  # orientações
    ax2 = fig.add_subplot(gs[1], sharex=ax1)  # número de eventos

    for ye_num in semesters_years:
        # Converte para Period para filtrar
        df_sta_year = df_sta[df_sta['year_semester'] == ye_num]

        ye_num = semestre_to_num[ye_num]

        if df_sta_year[df_sta_year['quality'] == 'good']['theta'].empty:
            orientations_bad = df_sta_year[df_sta_year['quality'] == 'bad']['clock_error'].values
            snr_bad = df_sta_year[df_sta_year['quality'] == 'bad']['SNR'].abs().values

            a = ax1.scatter([ye_num]*len(orientations_bad), orientations_bad, marker='.', c='gray',s=snr_bad*10, alpha=0.05, ec='k', label='bad')
            ax2.bar(ye_num, len(orientations_bad), color='gray', width=0.5, alpha=0.5)
        else:
            orientations_good = df_sta_year[df_sta_year['quality'] == 'good']['clock_error'].values
            orientations_bad = df_sta_year[df_sta_year['quality'] == 'bad']['clock_error'].values
            snr_good = df_sta_year[df_sta_year['quality'] == 'good']['SNR'].abs().values
            snr_bad = df_sta_year[df_sta_year['quality'] == 'bad']['SNR'].abs().values

            mean_sta = round(np.mean(orientations_good), 1)
            std_sta = round(np.std(orientations_good), 2)

            
            ax1.scatter([ye_num]*len(orientations_bad), orientations_bad, marker='.', c='gray',s=snr_bad*10, alpha=0.05, ec='k', label='bad')
            b = ax1.scatter([ye_num]*len(orientations_good), orientations_good, marker='.', c='k',s=snr_good*10, alpha=0.5, ec='k', label='good')

            ax1.annotate(f'{mean_sta}±{abs(std_sta)}°', (ye_num, 60), fontsize=10, rotation=90,va='center', ha='center',bbox=dict(boxstyle="round", fc="white", ec='k', alpha=0.5))
            ax1.boxplot(orientations_good, positions=[ye_num], sym='', showmeans=False, widths=0.25, bootstrap=10000)

            ax2.bar(ye_num, len(orientations_bad), color='gray', width=0.5, alpha=0.5)
            ax2.bar(ye_num, len(orientations_good), color='k', edgecolor='k', width=0.1)

    ax1.set_ylabel('Clock error (s)')
    ax1.set_xlabel('YYYY-SS')
    ax1.set_ylim(-110,110)
    ax1.yaxis.set_major_locator(MultipleLocator(20))
    ax1.yaxis.set_minor_locator(MultipleLocator(10))
    ax1.grid(True)
    ax1.tick_params(axis="x", which='both', labelbottom=False, labeltop=True, rotation=90)
    ax1.tick_params(axis="y", which='both', labelright=True, labelleft=True, left=True, right=True)
    ax1.set_title(f'{net}.{sta}', fontsize=20)
    ax1.legend(handles=[a,b])
    
    # Depois de plotar todos, ajustar o eixo X
    ax2.set_xticks(list(semestre_to_num.values()))
    ax2.set_xticklabels(list(semestre_to_num.keys()))
    ax2.yaxis.set_major_locator(MultipleLocator(15))
    ax2.tick_params(axis="x", which='both', labelbottom=True, labeltop=False, rotation=90)
    ax2.tick_params(axis="y", which='both', labelright=True, labelleft=True, left=True, right=True)
    ax2.set_ylim(0, 50)
    ax2.set_ylabel("n")
    ax2.grid(True)

    # Salvando a figura
    output_figure_ORIENTATION = ORIENTATION_OUTPUT+'TIMING_FIGURES/FINAL_RESULT/'
    os.makedirs(output_figure_ORIENTATION,exist_ok=True)
    fig.savefig(output_figure_ORIENTATION+'TIMING_TOTAL_'+sta+'.png',dpi=300)
    plt.close()

In [ ]:
# Lista de markers que aceitam edgecolor='k'
markers = ['o', 's', 'D', '^', 'v', '<', '>', 'p', '*', 'h', 'H']

# Lista de cores (você pode personalizar essas cores)
colors = plt.get_cmap('Accent').colors  # Paleta 'tab20' do Matplotlib

# Define o tamanho desejado
tamanho = len(STATION_LST)

# Gera a lista de markers e cores com repetição se necessário
marker_list = [markers[i % len(markers)] for i in range(tamanho)]
color_list = [colors[i % len(colors)] for i in range(tamanho)]

In [ ]:
plt.rcParams.update({'font.size': 14})  # Define o tamanho global da fonte

# Criando a figura
fig = plt.figure(figsize=(10, 10))
gs = gridspec.GridSpec(1, 1)
years = np.arange(2013, 2026, 1)

# Eixo 1: Medidas de orientação
ax1 = fig.add_subplot(gs[0])

for ista, sta in enumerate(tqdm(STATION_LST, total=len(STATION_LST), desc='Station')):
    df_sta = station_df[station_df['station'] == sta]

    YEAR_min = min(years)
    YEAR_max = max(years)

    # Flag para adicionar a legenda apenas no primeiro ponto de cada estação
    added_legend = False

    for idx, ye in enumerate(years):
        df_sta_year = df_sta[df_sta['year'] == int(ye)]

        if not df_sta_year[df_sta_year['quality'] == 'good']['theta'].empty:
            orientations_good = df_sta_year[df_sta_year['quality'] == 'good']['theta'].values
            snr_good = df_sta_year[df_sta_year['quality'] == 'good']['SNR'].abs().values

            # Plota com a legenda apenas no primeiro ponto
            ax1.scatter(
                ye, round(circmean(orientations_good, high=360, low=-360),1),
                marker=marker_list[ista], c=color_list[ista],
                s=snr_good.mean() * 10, alpha=0.5,ec='k',
                label=sta if not added_legend else None
            )
            added_legend = True  # Marca que a legenda foi adicionada

# Configurações do eixo
ax1.set_ylabel(r'Orientation($\theta$)')
ax1.set_xlabel('Year')
ax1.set_ylim(-180, 180)
ax1.xaxis.set_major_locator(MultipleLocator(2))
ax1.xaxis.set_minor_locator(MultipleLocator(1))
ax1.yaxis.set_major_locator(MultipleLocator(40))
ax1.yaxis.set_minor_locator(MultipleLocator(10))
ax1.yaxis.set_major_formatter(FuncFormatter(format_y_ticks))
ax1.grid(True)
ax1.tick_params(axis="both", labelbottom=True, labelright=True, labelleft=True, labeltop=True)
ax1.legend(loc='lower right',ncol=int(len(STATION_LST)/4))
ax1.set_title('Orientation Compilation', fontsize=20)

# Salvando a figura
output_figure_ORIENTATION = ORIENTATION_OUTPUT + 'ORIENTATION_FIGURES/FINAL_RESULT/'
os.makedirs(output_figure_ORIENTATION, exist_ok=True)
fig.savefig(output_figure_ORIENTATION + 'ORIENTATION_TOTAL_COMPILATION.png', dpi=300)
plt.close()


In [ ]:
plt.rcParams.update({'font.size': 15})  # Define o tamanho global da fonte

# Criando a figura
fig = plt.figure(figsize=(15, 10))
gs = gridspec.GridSpec(1, 1)
years = np.arange(2013, 2026, 1)

# Eixo 1: Medidas de orientação
ax1 = fig.add_subplot(gs[0])
ax1.axhline(y=0, xmin=0, xmax=1,c='k',ls=':',zorder=-10)

for ista, sta in enumerate(tqdm(STATION_LST, total=len(STATION_LST), desc='Station')):
    df_sta = station_df[station_df['station'] == sta]

    YEAR_min = min(years)
    YEAR_max = max(years)

    # Flag para adicionar a legenda apenas no primeiro ponto de cada estação
    added_legend = False

    for idx, ye in enumerate(years):
        df_sta_year = df_sta[df_sta['year'] == int(ye)]

        if not df_sta_year[df_sta_year['quality'] == 'good']['theta'].empty:
            orientations_good = df_sta_year[df_sta_year['quality'] == 'good']['theta'].values
            snr_good = df_sta_year[df_sta_year['quality'] == 'good']['SNR'].abs().values

            # Plota com a legenda apenas no primeiro ponto
            ax1.scatter(
                ye, round(circmean(orientations_good, high=360, low=-360),1),
                marker=marker_list[ista], c=color_list[ista],
                s=snr_good.mean() * 10, alpha=0.75,ec='k',linewidths=2,
                label=sta if not added_legend else None
            )


            added_legend = True  # Marca que a legenda foi adicionada

# Configurações do eixo
ax1.set_ylabel(r'Orientação ($\theta$)',fontsize=20)
ax1.set_xlabel('Ano',fontsize=20)
ax1.set_ylim(-180, 180)
ax1.xaxis.set_major_locator(MultipleLocator(2))
ax1.xaxis.set_minor_locator(MultipleLocator(1))
ax1.yaxis.set_major_locator(MultipleLocator(40))
ax1.yaxis.set_minor_locator(MultipleLocator(10))
ax1.yaxis.set_major_formatter(FuncFormatter(format_y_ticks))
ax1.grid(which='major',linestyle=':')
ax1.tick_params(axis="both", which='both',labelbottom=True, labelright=True, labelleft=True, labeltop=False,bottom=True, top=True, left=True, right=True)
ax1.legend(loc='lower right',ncol=int(len(STATION_LST)/4))
ax1.set_title('Rede RSIS: Avaliação das Orientações', y=1.05,fontsize=25, fontweight='bold')

# Salvando a figura
output_figure_ORIENTATION = ORIENTATION_OUTPUT + 'ORIENTATION_FIGURES/FINAL_RESULT/'
os.makedirs(output_figure_ORIENTATION, exist_ok=True)
fig.savefig(output_figure_ORIENTATION + 'ORIENTATION_TOTAL_COMPILATION.png', dpi=300)
plt.close()
